<a href="https://colab.research.google.com/github/ameerfayiz/ML-notebooks/blob/main/Mistral%207%20llama%20index%20Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q transformers
!pip install -q llama-index
!pip install -q llama-index-llms-llama-cpp
!pip install -q sentence-transformers
!pip install -q llama-index-embeddings-langchain
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 6.5 MB/s eta 0:00:00


In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 257.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 174.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.57-cp310-cp310-manylinux_2_35_x86_64.whl size=26393643 sha256=cfd4d85b52cd5f6e9dae61d9c4b1bbf0e81f906377c2261b7cd762b90e9c66c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-ye79aa8t/wheels/7e/c0/00/e98d6e198f941c623da37b3f674354cbdccfcfb2cb9cf1133d
Successfully built llama-cpp-python


In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate

In [8]:
import torch

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


total size (MB): 4368.44


4167it [00:33, 124.76it/s]                          
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /tmp/llama_index/models/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u3

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [13]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-13-7e47785a9da0>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [14]:
documents = SimpleDirectoryReader("/content/Data/").load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [15]:
query_engine = index.as_query_engine()
response = query_engine.query("which syrup have Chlorpheniramine Maleate in it")


llama_print_timings:        load time =     875.29 ms
llama_print_timings:      sample time =      39.66 ms /    56 runs   (    0.71 ms per token,  1412.07 tokens per second)
llama_print_timings: prompt eval time =    1242.28 ms /   758 tokens (    1.64 ms per token,   610.17 tokens per second)
llama_print_timings:        eval time =    1508.08 ms /    55 runs   (   27.42 ms per token,    36.47 tokens per second)
llama_print_timings:       total time =    3036.24 ms /   813 tokens


In [16]:
print(response)

 Based on the provided context information, the syrup that contains Chlorpheniramine Maleate in it is Alex Junior Syrup with the file path "/content/Data/sampl_med.txt" and the drug code "165376".


In [17]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

list all medicine with Aceclofenac in it. bullet points


Llama.generate: prefix-match hit

llama_print_timings:        load time =     875.29 ms
llama_print_timings:      sample time =      33.06 ms /    61 runs   (    0.54 ms per token,  1845.24 tokens per second)
llama_print_timings: prompt eval time =     968.63 ms /   706 tokens (    1.37 ms per token,   728.86 tokens per second)
llama_print_timings:        eval time =    1674.92 ms /    60 runs   (   27.92 ms per token,    35.82 tokens per second)
llama_print_timings:       total time =    2843.20 ms /   766 tokens


 * Acenext P 100mg/325mg Tablet by Cadila Pharmaceuticals Ltd
* Acenext P 100mg/325mg Tablet by Cadila Pharmaceuticals Ltd (strip of 10 tablets)
which medicin is this ? "Amaryl"


Llama.generate: prefix-match hit

llama_print_timings:        load time =     875.29 ms
llama_print_timings:      sample time =       7.86 ms /    14 runs   (    0.56 ms per token,  1782.08 tokens per second)
llama_print_timings: prompt eval time =     961.17 ms /   697 tokens (    1.38 ms per token,   725.16 tokens per second)
llama_print_timings:        eval time =     372.97 ms /    13 runs   (   28.69 ms per token,    34.86 tokens per second)
llama_print_timings:       total time =    1384.14 ms /   710 tokens


 The medicine mentioned in the context information is "Amaryl".
full name of "Amaryl"


...

Llama.generate: prefix-match hit

llama_print_timings:        load time =     875.29 ms
llama_print_timings:      sample time =      15.82 ms /    28 runs   (    0.56 ms per token,  1770.02 tokens per second)
llama_print_timings: prompt eval time =     927.88 ms /   692 tokens (    1.34 ms per token,   745.79 tokens per second)
llama_print_timings:        eval time =     766.10 ms /    27 runs   (   28.37 ms per token,    35.24 tokens per second)
llama_print_timings:       total time =    1791.10 ms /   719 tokens


 Based on the provided context information, the medicine that starts with "Aquasol" is Aquasol A Capsule.
which medicine name is closer to "Ascoril exp"


Llama.generate: prefix-match hit

llama_print_timings:        load time =     875.29 ms
llama_print_timings:      sample time =      24.42 ms /    43 runs   (    0.57 ms per token,  1760.92 tokens per second)
llama_print_timings: prompt eval time =     912.68 ms /   676 tokens (    1.35 ms per token,   740.68 tokens per second)
llama_print_timings:        eval time =    1187.02 ms /    42 runs   (   28.26 ms per token,    35.38 tokens per second)
llama_print_timings:       total time =    2249.47 ms /   718 tokens


 Based on the provided context information, the closest medicine name to "Ascoril exp" would be "Ascoril Plus Expectorant" with the code "395727".


KeyboardInterrupt: Interrupted by user